In [21]:
cd ~/cg1/Golitzin/SWEMLv2.0/Dataprocessing

/uufs/chpc.utah.edu/common/home/civil-group1/Golitzin/SWEMLv2.0/Dataprocessing


In [22]:
import numpy as np
import pandas as pd
import ee #pip install earthengine-api
import geemap
import EE_funcs
import os
from tqdm import tqdm
from tqdm.notebook import tqdm_notebook
import concurrent.futures as cf
import pyarrow as pa
import pyarrow.parquet as pq
import pickle as pkl
ee.Authenticate()
ee.Initialize()
import warnings
import boto3
warnings.filterwarnings("ignore")

In [23]:
#load access key
#HOME = os.getcwd()
HOME = os.chdir('..')
HOME = os.getcwd()
#HOME = os.path.expanduser('~')
KEYPATH = "AWSaccessKeys.csv"
print(f"{HOME}/{KEYPATH}")

if os.path.isfile(f"{HOME}/{KEYPATH}") == True:
    ACCESS = pd.read_csv(f"{HOME}/{KEYPATH}")

    #start session
    SESSION = boto3.Session(
        aws_access_key_id=ACCESS['Access key ID'][0],
        aws_secret_access_key=ACCESS['Secret access key'][0],
    )
    S3 = SESSION.resource('s3')
    #AWS BUCKET information
    BUCKET_NAME = 'national-snow-model'
    #S3 = boto3.resource('S3', config=Config(signature_version=UNSIGNED))
    BUCKET = S3.Bucket(BUCKET_NAME)
else:
    print("no AWS credentials present, skipping")

/uufs/chpc.utah.edu/common/home/civil-group1/Golitzin/SWEMLv2.0/AWSaccessKeys.csv


In [3]:
ModuleDir = os.getcwd()
ModuleDir = os.chdir('Dataprocessing')
ModuleDir = os.getcwd()

import get_Precip

In [4]:
region='Northwest'
output_res=1000
meta_path = f"{HOME}/data/TrainingDFs/{region}/{output_res}M_Resolution/{region}_metadata.parquet"
# ASO_meta_path = f"{HOME}/data/TrainingDFs/{region}/{output_res}M_Resolution/ASO_meta.parquet"
meta = pd.read_parquet(meta_path)
# ASO_meta = pd.read_parquet(ASO_meta_path)

In [5]:
lat,long = meta[['cen_lat','cen_lon']].median()
location = ee.Geometry.Point(long,lat)

In [17]:
print(lat,long)

47.717357947477524 -123.52321734101645


In [33]:
startdate='2015-10'
enddate='2019-07'
precip = ee.ImageCollection('NASA/NLDAS/FORA0125_H002').select('total_precipitation').filterDate(startdate, enddate)

In [34]:
precip_poi = precip.getRegion(location,scale=1000).getInfo()

In [35]:
site_precip = EE_funcs.ee_array_to_df(precip_poi,['total_precipitation'])

In [36]:
temporal_resample = 'D'
kgm2_to_cm = 0.1

site_precip.set_index('datetime', inplace = True)
site_precip = site_precip.resample(temporal_resample).sum()
site_precip.reset_index(inplace = True)

        #make columns for cms
site_precip['total_precipitation'] = site_precip['total_precipitation']*kgm2_to_cm
site_precip.rename(columns={'total_precipitation':'daily_precipitation_cm'}, inplace = True)
site_precip.pop('time')
site_precip.set_index('datetime',inplace=True)


In [15]:
WYdict = {2016,2019}

,daily_precipitation_cm
datetime,
2017-09-10,0.06440
2017-09-11,0.00000
2017-09-12,0.00000
2017-09-13,0.00000
2017-09-14,0.00000
2017-09-15,0.00000
2017-09-16,0.00000
2017-09-17,0.85516
2017-09-18,0.81654


In [27]:
precip_daymet = ee.ImageCollection('NASA/ORNL/DAYMET_V4').select('prcp').filterDate(startdate, enddate)
precip_daymet_poi = precip_daymet.getRegion(location,scale=1000).getInfo()

In [32]:
site_precip_daymet = EE_funcs.ee_array_to_df(precip_daymet_poi,['prcp'])

In [39]:
site_precip_daymet['prcp'] /= 10
site_precip_daymet.pop('time')
site_precip_daymet.set_index('datetime',inplace=True)
site_precip_daymet.rename(columns={'prcp':'daily_precipitation_cm'},inplace=True)

In [40]:
site_precip_daymet

,daily_precipitation_cm
datetime,
2015-10-01,0.00
2015-10-02,0.00
2015-10-03,0.00
2015-10-04,0.00
2015-10-05,0.00
...,...
2019-06-26,0.00
2019-06-27,0.63
2019-06-28,0.00


Map(center=[47.717357947477524, -123.52321734101645], controls=(WidgetControl(options=['position', 'transparen…

Map(center=[47.717357947477524, -123.52321734101645], controls=(WidgetControl(options=['position', 'transparen…